In [44]:
from dezero import Model
import dezero.functions as F
import dezero.layers as L


In [58]:
from  dezero.core import *

In [30]:
from dezero.layers import Layer

In [31]:
class RNN(Layer):
    def __init__(self, hidden_size, in_size=None):
        super().__init__()
        self.x2h = Linear(hidden_size, in_size=in_size)
        self.h2h = Linear(hidden_size, in_size=in_size, nobias = True)
        self.h = None
        
        
    def reset_state(self):
        self.h = None
        
    def forward(self, x):
        if self.h is None:
            h_new = F.tanh(self.x2h(x))
            
        else:
            h_new = F.tanh(self.x2h(x) + self.h2h(self.h))
            
        self.h = h_new
        return h_new
    

In [32]:
class SimpleRNN(Model):
    def __init__(self, hidden_size, out_size):
        super().__init__()
        self.rnn = L.RNN(hidden_size)
        self.fc = L.Linear(out_size)
    
    def reset_state(self):
        self.rnn.reset_state()
        
    def forward(self, x):
        h = self.rnn(x)
        y = self.fc(h)
        return y
    


In [33]:
seq_data = [np.random.randn(1,1) for _ in range(1000)] # 더미 데이터
xs = seq_data[0:-1]
ts = seq_data[1:]

model = SimpleRNN(10,1)

loss, cnt = 0, 0
for x ,t in zip(xs, ts):
    y = model(x)
    loss += F.mean_squared_error(y,t)
    
    cnt += 1
    
    if  cnt ==2: # 입력 두개째에서 끊어 버리기 위해서 TBPTT
        model.cleargrads()
        loss.backward()
        break

- 앞의 코드의 cnt = 2 입력 데이터를 끊어주기 위한 장치 입니다. 

- 물론 입력 데이터는 10, , 100개든 원하는 만큼 제공할 수 있습니다. 몇 개가 입력되든 계산 그래프는 문제없이 계속 길게 뻗어갑니다. 하지만 역전파를 잘하려면 계산 그래프를 적당한 길이에서 끊어 줘야 합니다. 이것이  Truncated BPTT  입니다. 

- 완전 연결 Affine 계층
- cnn 에서는 풀링 계층이 추가 됩니다. 'Conv-ReLU-(Pooling)'흐름으로 연결됩니다.


In [ ]:
- 완전 계층의 문제점
 - 데이터의 형상이 무시됨.
- 완전 계층을 시킨다면 이미지의 경우 3차원 데이터를 평평한 1차원 데이터로 바꿔줘야 합니다. 

- 이미지는 3차원 형상이며, 이 형사에는 소중한 공간적 정보가 담겨 있습니다. 예를 들어 공간적으로 가까운 픽셀은 값이 비슷하거나,
- 거리가 먼 픽셀끼리는 별 연관이 없는 등, 3차원 속에서 의미를 갖는 본질적ㅇ니 패턴이 숨어 있을 겁니다.

In [40]:
import numpy as np
import dezero.functions as F

x1 = np.random.rand(1,3, 7,7)
col1 = F.im2col(x1, kernel_size =5, stride=1, pad=0, to_matrix=True)
print(col1.shape)

x2 = np.random.rand(10,3, 7,7)
kernel_size = (5,5) # int 5 와 같은것임
stride= (1,1)
pad = (0,0)
col2 = F.im2col(x2, kernel_size, stride, pad, to_matrix=True)
print(col2.shape)



(9, 75)
(90, 75)


In [41]:
def pair(x):
    if isinstance(x, int):
        return (x,x)
    elif isinstance(x, tuple):
        assert len(x) == 2
        return x
    else:
        raise ValueError
        

In [54]:
from dezero.utils import pair, get_conv_outsize

def conv2d_simple(x, W, b=None, stride=1, pad=0):
    x, W = as_variable(x), as_variable(W)
    
    Weight = W # width 의 W와 헷갈리지 않기 위해서
    N, C, H, W = x.shape
    SH, SW = pair(stride)
    PH, PW = pair(pad)
    OH = get_conv_outsize(H,KH,SH,PH)
    OW = get_conv_outsize(W, KW, SW, PW)
    
    col = i2col(x, (KH,KW), stied, pad, to_matrix=True)
    Weight = Weigh.reshape(OC, -1).transpose()
    t = linear(col, Weight, b)
    y = t.reshape(N, OH, OW, OC).transpose(0, 3, 1, 2)
    return y

In [59]:
N, C, H, W = 1, 5, 15, 15
OC, (KH, KW) = 8, (3,3)

x = Variable(np.random.randn(N,C,H,W))
W = np.random.randn(OC, C, KH, KW)
y = F.conv2d_simple(x, W, b=None, stride=1, pad=1)
y.backward()

print(y.shape)
print(x.grad.shape)

(1, 8, 15, 15)
(1, 5, 15, 15)
